In [7]:
import heapq
import random

# Goal state
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

# Move directions: up, down, left, right
moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

class PuzzleState:
    def __init__(self, board, g, parent=None):
        self.board = board
        self.g = g
        self.h = self.calculate_heuristic()
        self.f = self.g + self.h
        self.parent = parent

    def __lt__(self, other):
        return self.f < other.f

    def calculate_heuristic(self):
        distance = 0
        for i in range(3):
            for j in range(3):
                val = self.board[i][j]
                if val != 0:
                    goal_x = (val - 1) // 3
                    goal_y = (val - 1) % 3
                    distance += abs(goal_x - i) + abs(goal_y - j)
        return distance

    def get_neighbors(self):
        neighbors = []
        x = y = 0
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    x, y = i, j
                    break

        for dx, dy in moves:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = [row[:] for row in self.board]
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                neighbors.append(PuzzleState(new_board, self.g + 1, self))
        return neighbors

    def is_goal(self):
        return self.board == goal_state

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(str(self.board))


def is_solvable(board):
    flat = [cell for row in board for cell in row if cell != 0]
    inversions = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inversions += 1
    return inversions % 2 == 0


def generate_solvable_board():
    tiles = list(range(9))  # 0–8
    while True:
        random.shuffle(tiles)
        board = [tiles[i:i+3] for i in range(0, 9, 3)]
        if is_solvable(board):
            return board


def a_star(start_board):
    start_state = PuzzleState(start_board, 0)
    open_set = []
    heapq.heappush(open_set, start_state)
    closed_set = set()

    while open_set:
        current = heapq.heappop(open_set)

        if current.is_goal():
            return reconstruct_path(current)

        closed_set.add(str(current.board))

        for neighbor in current.get_neighbors():
            if str(neighbor.board) not in closed_set:
                heapq.heappush(open_set, neighbor)

    return None


def reconstruct_path(state):
    path = []
    while state:
        path.append(state)
        state = state.parent
    return path[::-1]


# Main
if __name__ == "__main__":
    # Generate random solvable puzzle
    initial_state = generate_solvable_board()
    print("Generated solvable puzzle:\n")
    for row in initial_state:
        print(row)
    print()

    solution = a_star(initial_state)

    if solution:
        print(f"Solution found in {len(solution) - 1} moves:\n")
        for i, step in enumerate(solution):
            print(f"Step {i}:\n{step}\n")
    else:
        print("No solution found.")


Generated solvable puzzle:

[6, 8, 5]
[7, 4, 2]
[0, 3, 1]

Solution found in 26 moves:

Step 0:
6 8 5
7 4 2
0 3 1

Step 1:
6 8 5
0 4 2
7 3 1

Step 2:
6 8 5
4 0 2
7 3 1

Step 3:
6 8 5
4 2 0
7 3 1

Step 4:
6 8 0
4 2 5
7 3 1

Step 5:
6 0 8
4 2 5
7 3 1

Step 6:
0 6 8
4 2 5
7 3 1

Step 7:
4 6 8
0 2 5
7 3 1

Step 8:
4 6 8
2 0 5
7 3 1

Step 9:
4 6 8
2 3 5
7 0 1

Step 10:
4 6 8
2 3 5
7 1 0

Step 11:
4 6 8
2 3 0
7 1 5

Step 12:
4 6 0
2 3 8
7 1 5

Step 13:
4 0 6
2 3 8
7 1 5

Step 14:
4 3 6
2 0 8
7 1 5

Step 15:
4 3 6
2 1 8
7 0 5

Step 16:
4 3 6
2 1 8
7 5 0

Step 17:
4 3 6
2 1 0
7 5 8

Step 18:
4 3 0
2 1 6
7 5 8

Step 19:
4 0 3
2 1 6
7 5 8

Step 20:
4 1 3
2 0 6
7 5 8

Step 21:
4 1 3
0 2 6
7 5 8

Step 22:
0 1 3
4 2 6
7 5 8

Step 23:
1 0 3
4 2 6
7 5 8

Step 24:
1 2 3
4 0 6
7 5 8

Step 25:
1 2 3
4 5 6
7 0 8

Step 26:
1 2 3
4 5 6
7 8 0



In [3]:
import heapq

# Goal state (solved puzzle)
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

# Directions for moving the blank tile: (up, down, left, right)
moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Puzzle class to store each state
class Puzzle:
    def __init__(self, board, g, parent=None):
        self.board = board
        self.g = g  # cost so far
        self.h = self.heuristic()
        self.f = self.g + self.h
        self.parent = parent

    def __lt__(self, other):
        return self.f < other.f

    def heuristic(self):
        # Manhattan distance heuristic
        distance = 0
        for i in range(3):
            for j in range(3):
                val = self.board[i][j]
                if val != 0:
                    goal_x = (val - 1) // 3
                    goal_y = (val - 1) % 3
                    distance += abs(goal_x - i) + abs(goal_y - j)
        return distance

    def get_neighbors(self):
        neighbors = []
        x, y = 0, 0

        # Find blank tile (0)
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    x, y = i, j

        # Try all 4 directions
        for dx, dy in moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_board = [row[:] for row in self.board]
                new_board[x][y], new_board[nx][ny] = new_board[nx][ny], new_board[x][y]
                neighbors.append(Puzzle(new_board, self.g + 1, self))
        return neighbors

    def is_goal(self):
        return self.board == goal_state

    def __str__(self):
        return "\n".join(str(row) for row in self.board)

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(str(self.board))

def a_star(start_board):
    start = Puzzle(start_board, 0)
    open_set = []
    heapq.heappush(open_set, start)
    visited = set()

    while open_set:
        current = heapq.heappop(open_set)

        if current.is_goal():
            return reconstruct_path(current)

        visited.add(str(current.board))

        for neighbor in current.get_neighbors():
            if str(neighbor.board) not in visited:
                heapq.heappush(open_set, neighbor)

    return None

def reconstruct_path(state):
    path = []
    while state:
        path.append(state)
        state = state.parent
    return path[::-1]  # reverse to get path from start to goal

# Start state (change this for different puzzles)
start_state = [
    [1, 2, 3],
    [4, 0, 6],
    [7, 5, 8]
]

# Run the A* algorithm and print each step
if __name__ == "__main__":
    print("Start State:")
    for row in start_state:
        print(row)
    print()

    solution = a_star(start_state)

    if solution:
        for i, step in enumerate(solution):
            print(f"Step {i}:")
            print(step)
            print()
    else:
        print("No solution found.")


Start State:
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Step 0:
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Step 1:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step 2:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

